<a href="https://colab.research.google.com/github/priaJuli/project_thesis/blob/master/Prediction_Rainfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
textfiles = "/content/drive/MyDrive/Colab Notebooks/sharing collab/Copy of input1.txt"

In [3]:
import tensorflow as tf


from keras.optimizers import SGD, Adam
from keras.layers import Conv1D,Conv2D, MaxPool2D, MaxPool1D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed, InputLayer, Activation
from keras.models import Sequential
from keras import regularizers
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from keras.backend import clear_session as K_clear_session


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.test.gpu_device_name()

Num GPUs Available:  0


''

In [4]:
import pandas as pd
import numpy as np

In [5]:
!pip install -U scikit-learn

In [6]:
import sklearn
print(sklearn.__version__)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, train_test_split, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron, PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.ensemble import StackingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

1.3.1


In [7]:

import os

import math
import matplotlib.pyplot as plt
import requests
from IPython.display import Audio, display


# There is 2 version regression model

1. Model regression 0 -> Without Autocorrelation

2. Model regression 2 -> With autocorrelation

In [8]:

class Prediction_rainfall:
    def __init__(self, regresion0, regresion2, train_with_prec=False):
        self.format_datecol0 = "Date"
        self.format_datecol2 = "tanggal"
        self.sep_reg0 = ";"
        self.sep_reg2 = ","
        self.path_reg0 = regresion0
        self.path_reg2 = regresion2
        self.train_with_prec = train_with_prec

    def prepare_dataframe2(self):

        pd_data = pd.read_table(self.path_reg2, sep=self.sep_reg2)
        last_row = len(pd_data) - 1
        pd_data = pd_data.drop(pd_data.index[last_row])

        label = pd_data['prect']
        self.all_label2 = label

        self.dataframe_reg2_original = pd_data.copy()

        ts_cv = TimeSeriesSplit(
            n_splits=2,
            gap=20,
            max_train_size=int(len(pd_data)*0.9),
            test_size=1100,
        )
        all_splits = list(ts_cv.split(pd_data, label))
        self.ts_cv = ts_cv

        # Prepare 6 year data
        self.prepare_6year(all_splits[0][0], all_splits[0][1], self.dataframe_reg2_original, regresion=2)

        # Prepare 9 year data
        self.prepare_9year(all_splits[1][0], all_splits[1][1], self.dataframe_reg2_original, regresion=2)

        dropcols = self.prepare_col_drop(regresion=2)
        pd_data.drop(dropcols, inplace=True, axis=1)
        self.pd_data2 = pd_data

    def prepare_dataframe0(self):

        pd_data = pd.read_table(self.path_reg0, sep=self.sep_reg0)
        last_row = len(pd_data) - 1
        pd_data = pd_data.drop(pd_data.index[last_row])

        label = pd_data['prect']
        self.all_label0 = label

        self.dataframe_reg0_original = pd_data.copy()
        ts_cv = TimeSeriesSplit(
            n_splits=2,
            gap=20,
            max_train_size=int(len(pd_data)*0.9),
            test_size=1100,
        )
        self.ts_cv = ts_cv

        all_splits = list(ts_cv.split(pd_data, label))

        # Prepare 6 year data
        self.prepare_6year(all_splits[0][0], all_splits[0][1], self.dataframe_reg0_original, regresion=0)

        # Prepare 9 year data
        self.prepare_9year(all_splits[1][0], all_splits[1][1], self.dataframe_reg0_original, regresion=0)

        dropcols = self.prepare_col_drop(regresion=0)
        pd_data.drop(dropcols, inplace=True, axis=1)
        self.pd_data0 = pd_data

    def prepare_col_drop(self, regresion=0):
        datacoltodrop = self.format_datecol0 if regresion == 0 else self.format_datecol2
        if self.train_with_prec == True:
            return [ 'prect', datacoltodrop ]
        else:
            return ['prec', 'prect', datacoltodrop ]

    def prepare_6year(self, train_split, test_split, df_original, regresion=0):

        df_6th = pd.DataFrame(df_original.iloc[train_split])
        df_6th.reset_index(drop=True, inplace=True)
        if regresion == 0:
          self.df_6th0 = df_6th
        else:
          self.df_6th2 = df_6th

    def prepare_9year(self, train_split, test_split, df_original, regresion=0):

        df_9th = pd.DataFrame(df_original.iloc[train_split])
        df_9th.reset_index(drop=True, inplace=True)
        if regresion == 0:
          self.df_9th0 = df_9th
        else:
          self.df_9th2 = df_9th

    def GridSearch_pipeline_reg0(self, pipeline, param_grid=None, cv=None, saved_pipeline=True):
        ts_cv = cv if cv is not None else self.ts_cv

        df_treats_1, labelDF, labelOri = self.prepare_for_train(regresion=2)

        grid = GridSearchCV(pipeline, n_jobs=1, param_grid=param_grid, cv=ts_cv,
                    scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"], refit='neg_mean_absolute_error')
        grid.fit(df_treats_1, labelDF)
        # print(grid.cv_results_)
        print(-grid.cv_results_["mean_test_neg_mean_absolute_error"]  * np.max(labelOri))
        print(-grid.cv_results_["mean_test_neg_root_mean_squared_error"]  * np.max(labelOri))
        print(grid.best_params_)

        if saved_pipeline:
            self.pipeline0 = pipeline
            self.param_grid0 = param_grid
        else:
            self.pipeline0 = None
            self.param_grid0 = None


    def GridSearch_pipeline_reg2(self, pipeline, param_grid=None, cv=None, saved_pipeline=True):
        ts_cv = cv if cv is not None else self.ts_cv

        df_treats_1, labelDF, labelOri = self.prepare_for_train(regresion=2)

        grid = GridSearchCV(pipeline, n_jobs=1, param_grid=param_grid, cv=ts_cv,
                    scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"], refit='neg_mean_absolute_error')
        grid.fit(df_treats_1, labelDF)
        # print(grid.cv_results_)
        print(-grid.cv_results_["mean_test_neg_mean_absolute_error"]  * np.max(labelOri))
        print(-grid.cv_results_["mean_test_neg_root_mean_squared_error"]  * np.max(labelOri))
        print(grid.best_params_)

        if saved_pipeline:
            self.pipeline2 = pipeline
            self.param_grid2 = param_grid
        else:
            self.pipeline2 = None
            self.param_grid2 = None

    def prepare_for_train(self, regresion=0):

        if regresion == 0:
          copy_pd = self.pd_data0.copy()
          labelres = self.all_label0
        else:
          copy_pd = self.pd_data2.copy()
          labelres = self.all_label2

        if self.train_with_prec == True:
            copy_pd['prec'] = copy_pd['prec'] / np.max(copy_pd['prec'])

        labelscale = labelres / np.max(labelres)

        return copy_pd, labelscale, labelres

    def train_6year_with_one_year(self, year, regresion=0):
        if regresion == 0:
          df_treats_1 = self.dataframe_reg0_original
          datacoltodrop = self.format_datecol0
          df_train6year = self.df_6th0.copy()
          pipeline = self.pipeline0
          param_grid = self.param_grid0
          all_label = self.all_label0
        else:
          df_treats_1 = self.dataframe_reg2_original
          datacoltodrop = self.format_datecol2
          df_train6year = self.df_6th2.copy()
          pipeline = self.pipeline2
          param_grid = self.param_grid2
          all_label = self.all_label2


        df_extra_test = df_treats_1.copy()
        df_extra_test = df_extra_test[df_extra_test[datacoltodrop].str.contains(year)]
        labelextra_test = df_extra_test['prect']
        print("Label test original, max {}, min {}, average {}".format(np.max(labelextra_test), np.min(labelextra_test) , np.mean(labelextra_test) ))

        dropcols = self.prepare_col_drop(regresion=regresion)
        df_extra_test.drop(dropcols, inplace=True, axis=1)

        label6year = df_train6year['prect']
        maxlabel6year = np.max(label6year)

        print("Max label 6 year regression {} = {}".format(regresion, maxlabel6year ) )
        df_train6year.drop(dropcols, inplace=True, axis=1)

        label6year = label6year / np.max(maxlabel6year)
        labelextra_test = labelextra_test / np.max(maxlabel6year)

        pipeline.fit(df_train6year, label6year)
        y_pred_0 = pipeline.predict(df_extra_test)

        print("Label test predict, max {}, min {}, average {}".format(np.max(y_pred_0), np.min(y_pred_0) , np.mean(y_pred_0) ))
        y_pred_0 = y_pred_0 * np.max(maxlabel6year)

        print("Total test ", len(df_extra_test))
        print("RMSE", mean_squared_error(labelextra_test * np.max(maxlabel6year), y_pred_0, squared=False))

        print("MSE", mean_squared_error(labelextra_test * np.max(maxlabel6year), y_pred_0, squared=True))

        print("MAE", mean_absolute_error(labelextra_test * np.max(maxlabel6year), y_pred_0))


In [9]:
import pickle

Path files Malau

In [10]:
pathreg0 = "/content/drive/MyDrive/Colab Notebooks/sharing collab/Project join/regresi0.txt"
pathreg2 = "/content/drive/MyDrive/Colab Notebooks/sharing collab/Project join/regresi2.txt"

In [11]:
classnew = Prediction_rainfall(pathreg0, pathreg2, train_with_prec=False)

classnew.prepare_dataframe0()
classnew.prepare_dataframe2()

# The first year data 2010

In [12]:
classnew.df_6th2.head()

,tanggal,maxhum1,maxhum2,maxhum3,maxhum4,maxhum5,maxhum6,maxhum7,maxhum8,maxhum9,...,prec13,prec14,prec15,prec16,prec17,prec18,prec19,prec20,prec,prect
0,21/01/2010,99,95,91,92,94,96,90,92,90,...,3.0,0.0,4.4,25.1,0.5,73.4,0.0,0.2,0.0,4.6
1,22/01/2010,90,99,95,91,92,94,96,90,92,...,4.2,3.0,0.0,4.4,25.1,0.5,73.4,0.0,4.6,0.0
2,23/01/2010,95,90,99,95,91,92,94,96,90,...,0.2,4.2,3.0,0.0,4.4,25.1,0.5,73.4,0.0,15.3
3,24/01/2010,89,95,90,99,95,91,92,94,96,...,109.9,0.2,4.2,3.0,0.0,4.4,25.1,0.5,15.3,0.0
4,25/01/2010,96,89,95,90,99,95,91,92,94,...,1.8,109.9,0.2,4.2,3.0,0.0,4.4,25.1,0.0,46.1


# The last date of 6 year

In [13]:
classnew.df_6th2.tail()

,tanggal,maxhum1,maxhum2,maxhum3,maxhum4,maxhum5,maxhum6,maxhum7,maxhum8,maxhum9,...,prec13,prec14,prec15,prec16,prec17,prec18,prec19,prec20,prec,prect
2136,27/11/2015,86,85,79,76,81,84,92,90,85,...,0.0,1.1,30.7,1.0,14.3,0.0,0.0,26.5,0.0,1.6
2137,28/11/2015,84,86,85,79,76,81,84,92,90,...,0.0,0.0,1.1,30.7,1.0,14.3,0.0,0.0,1.6,25.0
2138,29/11/2015,88,84,86,85,79,76,81,84,92,...,30.5,0.0,0.0,1.1,30.7,1.0,14.3,0.0,25.0,0.0
2139,30/11/2015,92,88,84,86,85,79,76,81,84,...,42.2,30.5,0.0,0.0,1.1,30.7,1.0,14.3,0.0,0.0
2140,01/12/2015,88,92,88,84,86,85,79,76,81,...,0.0,42.2,30.5,0.0,0.0,1.1,30.7,1.0,0.0,0.0


# The last date of 9 year

In [14]:
classnew.df_9th2.tail()

,tanggal,maxhum1,maxhum2,maxhum3,maxhum4,maxhum5,maxhum6,maxhum7,maxhum8,maxhum9,...,prec13,prec14,prec15,prec16,prec17,prec18,prec19,prec20,prec,prect
3236,01/12/2018,95,99,93,95,92,90,97,81,81,...,0.0,0.0,26.0,0.0,1.7,0.0,12.0,0.0,0.0,40.2
3237,02/12/2018,89,95,99,93,95,92,90,97,81,...,0.0,0.0,0.0,26.0,0.0,1.7,0.0,12.0,40.2,71.2
3238,03/12/2018,95,89,95,99,93,95,92,90,97,...,0.0,0.0,0.0,0.0,26.0,0.0,1.7,0.0,71.2,6.0
3239,04/12/2018,100,95,89,95,99,93,95,92,90,...,0.0,0.0,0.0,0.0,0.0,26.0,0.0,1.7,6.0,2.3
3240,05/12/2018,94,100,95,89,95,99,93,95,92,...,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,2.3,0.0


# Original data last date

In [15]:
classnew.dataframe_reg0_original.tail()

,Date,maxtemp,mintemp,avgtemp,maxhum,minhum,avghum,maxwind,maxqff,minqff,avgqff,maxqfe,minqfe,avgqfe,prec,prect
4377,26/12/2021,32.3,24.4,26.7,94.0,67.0,85.0,15.0,1012.6,1009.1,1011.0,1012.2,1008.7,1010.6,6.0,36.9
4378,27/12/2021,31.6,23.6,26.0,95.0,63.0,87.0,13.0,1013.3,1009.6,1011.4,1012.9,1009.2,1011.0,36.9,3.5
4379,28/12/2021,32.0,24.5,27.1,91.0,63.0,81.0,14.0,1012.1,1007.9,1010.3,1011.7,1007.5,1009.9,3.5,0.0
4380,29/12/2021,30.0,24.6,27.7,88.0,65.0,79.0,11.0,1011.8,1007.9,1010.0,1011.5,1007.6,1009.7,0.0,4.7
4381,30/12/2021,31.5,24.6,27.4,94.0,65.0,83.0,12.0,1011.8,1007.2,1009.8,1011.5,1006.9,1009.5,4.7,67.9


# After drop column of train data

In [16]:
classnew.pd_data2.tail()

,maxhum1,maxhum2,maxhum3,maxhum4,maxhum5,maxhum6,maxhum7,maxhum8,maxhum9,maxhum10,...,prec11,prec12,prec13,prec14,prec15,prec16,prec17,prec18,prec19,prec20
4356,93,95,90,95,95,91,89,88,94,93,...,0.6,0.0,0.0,0.2,2.3,0.0,0.0,0.4,5.8,1.5
4357,93,93,95,90,95,95,91,89,88,94,...,5.5,0.6,0.0,0.0,0.2,2.3,0.0,0.0,0.4,5.8
4358,94,93,93,95,90,95,95,91,89,88,...,3.5,5.5,0.6,0.0,0.0,0.2,2.3,0.0,0.0,0.4
4359,95,94,93,93,95,90,95,95,91,89,...,0.0,3.5,5.5,0.6,0.0,0.0,0.2,2.3,0.0,0.0
4360,91,95,94,93,93,95,90,95,95,91,...,0.5,0.0,3.5,5.5,0.6,0.0,0.0,0.2,2.3,0.0


Other solution  

StandarScaler()

MinMaxScaler()

In [17]:
pipes4 = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("classify", MLPRegressor(hidden_layer_sizes=(4,), max_iter=200, solver='sgd', shuffle=False )),
    ]
)

activations_opt = ['relu']

param_grid2 = [
    {
        "classify__activation": activations_opt,
    }
]

classnew.GridSearch_pipeline_reg0(pipes4, param_grid=param_grid2, cv=None)

[14.53227624]
[19.78686961]
{'classify__activation': 'relu'}


Other solution  

StandarScaler()

MinMaxScaler()

In [18]:
pipes3 = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("classify", MLPRegressor(hidden_layer_sizes=(4,), max_iter=200, solver='sgd', shuffle=False )),
    ]
)

activations_opt = ['relu']

param_grid2 = [
    {
        "classify__activation": activations_opt,
    }
]

classnew.GridSearch_pipeline_reg2(pipes3, param_grid=param_grid2, cv=None)

[14.04070175]
[19.40913021]
{'classify__activation': 'relu'}


In [19]:
print("Year test 2019")
classnew.train_6year_with_one_year("2019", regresion=0)
print("")
print("Year test 2020")
classnew.train_6year_with_one_year("2020", regresion=0)
print("")
print("Year test 2021")
classnew.train_6year_with_one_year("2021", regresion=0)

Year test 2019
Label test original, max 53.1, min 0.0, average 3.6145205479452054
Max label 6 year regression 0 = 170.4
Label test predict, max 0.22852646278040867, min -0.20775711464136987, average 0.03865535374934178
Total test  365
RMSE 14.301447395665498
MSE 204.53139761098745
MAE 11.122030810810253

Year test 2020
Label test original, max 123.9, min 0.0, average 6.323224043715847
Max label 6 year regression 0 = 170.4
Label test predict, max 0.33635324722097837, min -0.08968877554436663, average 0.04446639254004041
Total test  366
RMSE 15.65720989093098
MSE 245.14822156866694
MAE 9.997682183230888

Year test 2021
Label test original, max 171.0, min 0.0, average 6.8609890109890115
Max label 6 year regression 0 = 170.4
Label test predict, max 0.3065366126548522, min -0.2392820456458358, average 0.05935938336914447
Total test  364
RMSE 23.26075180433517
MSE 541.062574502882
MAE 17.215716451727904


In [20]:
print("Year test 2019")
classnew.train_6year_with_one_year("2019", regresion=2)
print("")
print("Year test 2020")
classnew.train_6year_with_one_year("2020", regresion=2)
print("")
print("Year test 2021")
classnew.train_6year_with_one_year("2021", regresion=2)

Year test 2019
Label test original, max 53.1, min 0.0, average 3.6145205479452054
Max label 6 year regression 2 = 170.4
Label test predict, max 0.029109793491522387, min -0.03580555733521283, average 0.02865206371221409
Total test  365
RMSE 8.862588022107609
MSE 78.54546644960524
MAE 6.2410967032453675

Year test 2020
Label test original, max 123.9, min 0.0, average 6.323224043715847
Max label 6 year regression 2 = 170.4
Label test predict, max 0.44934442552445897, min -0.26148106098806223, average 0.07705890792480535
Total test  366
RMSE 25.412717080165933
MSE 645.8061893965574
MAE 19.933935870160816

Year test 2021
Label test original, max 171.0, min 0.0, average 6.692837465564739
Max label 6 year regression 2 = 170.4
Label test predict, max 0.0427573452889191, min 0.0427573452889191, average 0.04275734528891911
Total test  363
RMSE 15.978209685500824
MSE 255.30318475383234
MAE 9.277409326811355
